## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

In [3]:
print(tf.__version__)

2.1.0-rc1


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [5]:
from google.colab import files
uploaded = files.upload()

Saving prices.csv to prices.csv


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('prices.csv')

In [8]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [9]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(['date','symbol'],axis=1)

In [11]:
data.columns

Index(['open', 'close', 'low', 'high', 'volume'], dtype='object')

In [12]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data = data.iloc[0:1000,:]

In [14]:
data.shape

(1000, 5)

In [0]:
data['volume'] = (data['volume'])/1000000

In [16]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [0]:
X = data.drop("volume", axis=1)
y = data["volume"]

In [18]:
X.shape

(1000, 4)

In [19]:
y.shape

(1000,)

In [20]:
X.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [21]:
y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [24]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 822 to 175
Data columns (total 4 columns):
open     700 non-null float64
close    700 non-null float64
low      700 non-null float64
high     700 non-null float64
dtypes: float64(4)
memory usage: 27.3 KB


In [25]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 778 to 459
Data columns (total 4 columns):
open     300 non-null float64
close    300 non-null float64
low      300 non-null float64
high     300 non-null float64
dtypes: float64(4)
memory usage: 11.7 KB


#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = X_train.astype('float32')

In [0]:
X_test = X_test.astype('float32')

In [0]:
y_train=y_train.astype('float32')

In [0]:
y_test=y_test.astype('float32')

In [30]:
X_train.dtypes

open     float32
close    float32
low      float32
high     float32
dtype: object

In [31]:
y_train.dtype

dtype('float32')

In [32]:
X_test.dtypes

open     float32
close    float32
low      float32
high     float32
dtype: object

In [33]:
y_test.dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scaler=StandardScaler()

In [36]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
X_train_Scaler=scaler.transform(X_train)

In [0]:
X_test_Scaler=scaler.transform(X_test)

In [0]:
y_train=np.array(y_train)

In [0]:
y_test=np.array(y_test)

In [41]:
X_train_Scaler

array([[-0.5680221 , -0.5701456 , -0.5665494 , -0.5715895 ],
       [ 1.0561224 ,  1.048834  ,  1.0359256 ,  1.0442692 ],
       [-0.60972595, -0.613514  , -0.6075959 , -0.6132277 ],
       ...,
       [ 0.24248421,  0.23934416,  0.24889648,  0.2362578 ],
       [ 0.99826455,  0.9795764 ,  0.99138933,  0.9821396 ],
       [ 0.8987026 ,  0.88706803,  0.896002  ,  0.8942731 ]],
      dtype=float32)

In [42]:
X_test_Scaler

array([[-0.22515987, -0.24941677, -0.24415994, -0.23012318],
       [-0.15098296, -0.1511369 , -0.14544897, -0.15389568],
       [-0.63758343, -0.63231254, -0.63285524, -0.63650584],
       ...,
       [-0.11719128, -0.10562477, -0.11570271, -0.10242164],
       [-0.47604266, -0.47631797, -0.47082964, -0.48028043],
       [-0.8850047 , -0.883289  , -0.8811284 , -0.88600737]],
      dtype=float32)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(X_train_Scaler, W, b):
    y_predict = tf.add(tf.matmul(X_train_Scaler, W), b)
    return y_predict

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_train, prediction):
    diff = y_train - prediction
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(X, y, W, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([W,b])
        current_prediction = prediction(X_train_Scaler, W, b)
        current_loss = loss(y_train, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dW, db = t.gradient(current_loss,[W, b])
    
    # Update Weights and Bias
    W = W - learning_rate * dW
    b = b - learning_rate * db
    
    return W, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [47]:
for i in range(100):    
    W, b = train(X_train_Scaler,y_train, W, b)
    print('Current Training Loss on iteration', i, loss(y_train, prediction(X_train_Scaler, W, b)).numpy())

Current Training Loss on iteration 0 304.18265
Current Training Loss on iteration 1 302.98566
Current Training Loss on iteration 2 301.83594
Current Training Loss on iteration 3 300.7317
Current Training Loss on iteration 4 299.6714
Current Training Loss on iteration 5 298.65286
Current Training Loss on iteration 6 297.67484
Current Training Loss on iteration 7 296.73544
Current Training Loss on iteration 8 295.83347
Current Training Loss on iteration 9 294.967
Current Training Loss on iteration 10 294.13474
Current Training Loss on iteration 11 293.3357
Current Training Loss on iteration 12 292.56836
Current Training Loss on iteration 13 291.83115
Current Training Loss on iteration 14 291.12326
Current Training Loss on iteration 15 290.44327
Current Training Loss on iteration 16 289.79022
Current Training Loss on iteration 17 289.16324
Current Training Loss on iteration 18 288.56104
Current Training Loss on iteration 19 287.98242
Current Training Loss on iteration 20 287.42703
Current

### Get the shapes and values of W and b

In [51]:
W.shape

TensorShape([4, 1])

In [52]:
b.shape

TensorShape([1])

In [53]:
W.numpy()

array([[-2.4773177e-09],
       [-3.0053787e-08],
       [ 2.8256327e-08],
       [-6.4726904e-09]], dtype=float32)

In [54]:
b.numpy()

array([4.866219], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [55]:
prediction(X_train_Scaler, W, b)

<tf.Tensor: shape=(700, 1), dtype=float32, numpy=
array([[4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],
       [4.866219],


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [56]:
from google.colab import files
uploaded = files.upload()

Saving iris.csv to iris.csv


In [0]:
iris=pd.read_csv('iris.csv')

In [58]:
iris.head()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm),Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [59]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
Sepal Length (in cm)    150 non-null float64
Sepal Width in (cm)     150 non-null float64
Petal length (in cm)    150 non-null float64
Petal width (in cm)     150 non-null float64
Class                   150 non-null object
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
#from sklearn.preprocessing import LabelEncoder

In [0]:
#labelencoder=LabelEncoder()
#iris['Class']=labelencoder.fit_transform(iris['Class'])

In [0]:
#iris['Class'].unique()

In [0]:
#iris=pd.get_dummies(iris)

In [0]:
iris['Class']=pd.get_dummies(iris['Class'])

In [61]:
iris['Class'].unique()

array([1, 0], dtype=uint64)

In [62]:
iris.head()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm),Class
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


In [63]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
Sepal Length (in cm)    150 non-null float64
Sepal Width in (cm)     150 non-null float64
Petal length (in cm)    150 non-null float64
Petal width (in cm)     150 non-null float64
Class                   150 non-null uint8
dtypes: float64(4), uint8(1)
memory usage: 5.0 KB


In [64]:
iris.head()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm),Class
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


### Splitting the data into feature set and target set

In [0]:
X = iris.drop("Class", axis=1)
y = iris["Class"]

In [0]:
#X=iris.drop(['Class_Iris-setosa', 'Class_Iris-versicolor', 'Class_Iris-virginica'], axis=1)
#y =iris['Class_Iris-setosa','Class_Iris-versicolor','Class_Iris-virginica']

In [67]:
X.head()

,Sepal Length (in cm),Sepal Width in (cm),Petal length (in cm),Petal width (in cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [68]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: Class, dtype: uint8

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()

In [0]:
# Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

In [0]:
# Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
# Compile the model
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [73]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 3)                 15        
Total params: 31
Trainable params: 23
Non-trainable params: 8
_________________________________________________________________


### Model Training 

In [74]:
model.fit(X, y, epochs=50)

Train on 150 samples
Epoch 1/50
150/150 [==============================] - 0s 2ms/sample - loss: 1.6279 - accuracy: 0.0933
Epoch 2/50
150/150 [==============================] - 0s 66us/sample - loss: 1.5044 - accuracy: 0.1600
Epoch 3/50
150/150 [==============================] - 0s 72us/sample - loss: 1.3804 - accuracy: 0.2067
Epoch 4/50
150/150 [==============================] - 0s 72us/sample - loss: 1.2912 - accuracy: 0.2267
Epoch 5/50
150/150 [==============================] - 0s 69us/sample - loss: 1.2110 - accuracy: 0.2933
Epoch 6/50
150/150 [==============================] - 0s 68us/sample - loss: 1.1370 - accuracy: 0.3267
Epoch 7/50
150/150 [==============================] - 0s 66us/sample - loss: 1.0639 - accuracy: 0.4133
Epoch 8/50
150/150 [==============================] - 0s 67us/sample - loss: 1.0025 - accuracy: 0.4533
Epoch 9/50
150/150 [==============================] - 0s 61us/sample - loss: 0.9551 - accuracy: 0.5133
Epoch 10/50
150/150 [==============================] 

Model Prediction

In [0]:
predict=model.predict(X)

In [76]:
predict

array([[2.21108362e-01, 6.76544130e-01, 1.02347456e-01],
       [2.30856657e-01, 6.20200515e-01, 1.48942724e-01],
       [1.27690777e-01, 7.54924774e-01, 1.17384434e-01],
       [1.13859147e-01, 7.65529096e-01, 1.20611764e-01],
       [1.64347112e-01, 7.45437622e-01, 9.02152210e-02],
       [2.97915459e-01, 6.20980978e-01, 8.11035708e-02],
       [8.69702846e-02, 8.18813205e-01, 9.42165107e-02],
       [1.98996529e-01, 6.93463445e-01, 1.07540086e-01],
       [8.88005346e-02, 7.77850330e-01, 1.33349136e-01],
       [2.05136880e-01, 6.64784133e-01, 1.30078986e-01],
       [3.24477315e-01, 5.86066067e-01, 8.94566178e-02],
       [1.31035000e-01, 7.70923197e-01, 9.80417803e-02],
       [1.83095694e-01, 6.77762032e-01, 1.39142290e-01],
       [6.00157082e-02, 8.29437613e-01, 1.10546671e-01],
       [4.79257077e-01, 4.50638294e-01, 7.01047033e-02],
       [3.33286732e-01, 6.10293150e-01, 5.64201511e-02],
       [2.95546889e-01, 6.19671345e-01, 8.47817808e-02],
       [2.29729727e-01, 6.63346

In [77]:
results = model.evaluate(X, y)

print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

150/150 [==============================] - 0s 482us/sample - loss: 0.2252 - accuracy: 0.9867
Final test set loss: 0.225224
Final test set accuracy: 0.986667


### Save the Model

In [78]:
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?